In [2]:
import IPython.display as ipd
import numpy as np
import wave
import bokeh 
from bokeh.io import output_notebook
from bokeh.plotting import figure, output_file, show
from scipy.io import wavfile
import librosa
import boreal

#
def hz2radians(f, srate):
    return 2 * np.pi * f / srate

y,srate = librosa.load("Mug1.wav")
mag_spectrum = abs(np.fft.rfft(y))
p = figure(plot_width=600, plot_height=200)
freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
max_freq_bin = int(srate / len(mag_spectrum) * 2000)
p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
show(p)


In [3]:
y3,srate = librosa.load("src_notebooks_mug_example.wav")
mag_spectrum = abs(np.fft.rfft(y3))
p = figure(plot_width=600, plot_height=200)
freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
max_freq_bin = int(srate / len(mag_spectrum) * 2000)
p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
show(p)

In [4]:
ipd.Audio("src_notebooks_mug_example.wav", rate = srate)

In [5]:
ipd.Audio("Mug2.wav", rate=srate)

In [6]:
y2,srate = librosa.load("Mug2.wav")
mag_spectrum = abs(np.fft.rfft(y2))
p = figure(plot_width=600, plot_height=200)
freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
max_freq_bin = int(srate / len(mag_spectrum) * 2000)
p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
show(p)

In [7]:
import librosa
srate = 44100      # sampling rate 
duration = 1       # duration in seconds 
freq = 120         # frequency 

t = np.linspace(0,duration,srate*duration)     # time instances of samples 
h0 =  0.5 * np.sin(2*np.pi*freq*t+np.pi/4);    # fundamental
h1 = 0.0 *  np.sin(2*np.pi*2*freq*t);          # octave  
t = np.linspace(0,duration+1,srate*duration)  
h2 = 0.2 * np.sin(2*np.pi*3*freq*t);           # octave+fifth 
y,srate = librosa.load("Mug1.wav", duration=1.20515)
y2,srate = librosa.load("Mug2.wav")
print(len(y) , len(y2))
# play the corresponding audio 
data = 0.3 * (h0 + h1 + h2) 
data2 = (y + y2)
print(data.shape[0])
ipd.Audio(data2,rate=srate)

26574 26574
44100


In [10]:
def envelope(segments,srate,duration): 
    nsamples = int(srate*duration)
    value = 0.0
    segment_index = 0 
    data = np.zeros(nsamples)
    segment_sample = 0 
    prev_target = 0.0

    for i in np.arange(nsamples): 
        if (segment_index < len(segments)): 
            target = segments[segment_index][0]
            ramp_time = segments[segment_index][1]
            delay_time = segments[segment_index][2]
            
            ramp_samples = (ramp_time / 1000.0) * srate 
            delay_samples = (delay_time / 1000.0) * srate
            
            if i < segment_sample + ramp_samples: 
                incr = (target-prev_target) / ramp_samples 
            elif i < segment_sample + ramp_samples + delay_samples: 
                incr = 0.0 
            else: 
                if ramp_samples != 0.0: 
                    incr = (target-prev_target) / ramp_samples 
                else: 
                    incr = 0.0 
                segment_sample = i 
                segment_index = segment_index+1 
                prev_target = target 
            value = value + incr 
        data[i] = value
    return data  
    

s1 = [(1, 50, 0), (1,100,50), (0,200, 100)]
s2 = [(0.5, 50, 0), (0.3, 100, 0), (0.0, 200, 200)]
s3 = [(1.5, 50, 0), (0.5, 1000, 0), (0.0, 200, 200)]
s4 = [(1.0, 50, 0), (0.45, 100, 0), (0.0, 200, 200)]

def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data
srate = 44100
dur = 4.0 
f0 = 200 
penv1 = envelope(s1, srate, dur)
penv2 = envelope(s1, srate, dur)
penv3 = envelope(s1, srate, dur)
penv4 = envelope(s1, srate, dur)

osc1 = sinusoid(f0, dur=dur)
osc2 = sinusoid(2*f0, dur=dur)
osc3 = sinusoid(3*f0, dur=dur)
osc4 = sinusoid(4*f0, dur=dur)
data = 0.25*(penv1 * osc1 + penv2 * osc2 + penv3* osc3 + penv4 * osc4)


mag_spectrum = abs(np.fft.rfft(data))
p = figure(plot_width=600, plot_height=200)
freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
max_freq_bin = int(srate / len(mag_spectrum) * 2000)
p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
show(p)

ipd.Audio(data,rate=srate)

In [15]:
s1 = [(1, 50, 0), (1,200,50), (0,200, 400)]
s2 = [(0.5, 50, 0), (0.3, 200, 100), (0.0, 200, 400)]
s3 = [(1.5, 50, 0), (0.5, 200, 100), (0.0, 200, 400)]
s4 = [(1.0, 50, 0), (0.45, 200, 100), (0.0, 200, 400)]

def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data
srate = 44100
dur = 4.0 
f0 = 200 
penv1 = envelope(s1, srate, dur)
penv2 = envelope(s1, srate, dur)
penv3 = envelope(s1, srate, dur)
penv4 = envelope(s1, srate, dur)

osc1 = sinusoid(f0, dur=dur)
osc2 = sinusoid(2*f0, dur=dur)
osc3 = sinusoid(3*f0, dur=dur)
osc4 = sinusoid(4*f0, dur=dur)
data = 0.25*(penv1 * osc1 + penv2 * osc2 + penv3* osc3 + penv4 * osc4)

mag_spectrum = abs(np.fft.rfft(data))
p = figure(plot_width=600, plot_height=200)
freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
max_freq_bin = int(srate / len(mag_spectrum) * 2000)
p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
show(p)


In [11]:
import scipy.signal
import numpy as np
import IPython.display as ipd
import boreal

srate = 48000
N = int(1.5*srate)   # number of samples to compute

impulse = np.zeros(N)
impulse[0] = 1 

def modal_resonance(audio, amp, freq, radius, srate): 
    b = np.ones(1)
    a = np.zeros(3)
    a[0] = 1.0 
    a[1] = -2*radius * np.cos(2*np.pi*freq*(1.0/srate))
    a[2] = radius * radius 
    # apply filter
    filtered_audio = amp * scipy.signal.lfilter(b, a, audio)
    return filtered_audio 

def modal_note(fundamental, excitation, N): 
    # Mode parameters
    nModes = 4
    freqs = fundamental * np.array([0.3, 1.51,2.57, 4.54]) # modal center frequencies
    radii = [ 0.999, 0.9999, 0.9995, 0.9995] # modal radii
    amps = [40, 60, 50, 70]    
    
    modes = [] 
    mix = np.zeros(N)
    for m in np.arange(0, nModes): 
        modes.append(modal_resonance(excitation, amps[m], freqs[m], radii[m], srate))
        mix += modes[m]
        # normalize modes after mixing for individual playback 
        modes[m] = 0.5 * (modes[m] / np.max(modes[m]))
    mix = 0.5 * (mix / np.max(mix))
    return mix, modes 
    
fundamental = 800 
(mix1,modes) = modal_note(fundamental, impulse, N)
mag_spectrum = abs(np.fft.rfft(mix1))
p = figure(plot_width=600, plot_height=200)
freqs = np.linspace(0, 0.5 * srate, len(mag_spectrum))
max_freq_bin = int(srate / len(mag_spectrum) * 2000)
p.line(freqs[0:max_freq_bin],mag_spectrum[0:max_freq_bin] * 2 * (1.0 / srate))
show(p)

ipd.Audio(mix1, rate=srate,normalize=False)